In [2]:
import pandas as pd 

### Data
* mRNA_df (1408, 19194) | OmicsExpressionProteinCodingGenesTPMLogp1.csv
    * Depmap ID
* gene_dependency_df (1095, 17932) | CRISPRGeneDependency.csv
    * ModelID
* cells_df (103, 6) | Depmap Melanoma Cell Lines.csv
    * Depmap Id

In [3]:
# mRNA
mRNA_df = pd.read_csv(r'C:\Users\commo\OneDrive - University of Virginia\School\STEM\BME\BME 4700 - Systems Bioengineering\CCLE Chromatin Analysis\Raw Data\OmicsExpressionProteinCodingGenesTPMLogp1.csv')
print(mRNA_df.shape)


(1408, 19194)


In [4]:
# Gene Dependency 
gene_dependency_df = pd.read_csv(r'C:\Users\commo\OneDrive - University of Virginia\School\STEM\BME\BME 4700 - Systems Bioengineering\CCLE Chromatin Analysis\Raw Data\CRISPRGeneDependency.csv')
print(gene_dependency_df.shape)


(1095, 17932)


In [5]:
# Gene Effect
gene_effect_df = pd.read_csv(r'C:\Users\commo\OneDrive - University of Virginia\School\STEM\BME\BME 4700 - Systems Bioengineering\CCLE Chromatin Analysis\Raw Data\CRISPRGeneEffect.csv')
print(gene_effect_df.shape)

(1078, 17454)


In [6]:
# DepMap Melanome Cell Lines
cells_df = pd.read_csv(r'C:\Users\commo\OneDrive - University of Virginia\School\STEM\BME\BME 4700 - Systems Bioengineering\CCLE Chromatin Analysis\Raw Data\Depmap Melanoma Cell Lines.csv')
cells_df.drop(columns=['Unnamed: 0'], inplace=True)
print(cells_df.shape)


(103, 6)


### ID Matching

* 60 Cell Lines match in both, limited by gene dependency data

In [9]:
# Matches mRNA_df and gene_dependency_df
# SWAP THIS TO RUN GENE EFFECT
matches_df = pd.merge(mRNA_df, gene_effect_df, left_on='Depmap ID', right_on = 'Unnamed: 0', how='inner')
print(matches_df.shape)

(1003, 36648)


In [10]:
# Matches matches_df and cells_df
cell_matches_df = pd.merge(matches_df, cells_df, left_on='Depmap ID', right_on = 'Depmap Id', how='inner')
print(cell_matches_df.shape)

(60, 36654)


### Save Cell Matches

#### Preprocessing

In [11]:
# Index into mRNA by list(cell_matches_df['Depmap ID'])
depmap_mRNA_df = mRNA_df[mRNA_df['Depmap ID'].isin(cell_matches_df['Depmap ID'])]
print(depmap_mRNA_df.shape)

# Index into gene_dependency by list(cell_matches_df['Depmap ID'])
depmap_gene_dependency_df = gene_dependency_df[gene_dependency_df['ModelID'].isin(cell_matches_df['Depmap ID'])]
print(depmap_gene_dependency_df.shape)

# Index into gene_effect by list(cell_matches_df['Depmap ID'])
depmap_gene_effect_df = gene_effect_df[gene_effect_df['Unnamed: 0'].isin(cell_matches_df['Depmap ID'])]
print(depmap_gene_effect_df.shape)

(60, 19194)
(60, 17932)
(60, 17454)


In [12]:
# Remove (xxxx) from columns
depmap_gene_dependency_df.columns = depmap_gene_dependency_df.columns.str.replace(r" \(.*\)","")
depmap_mRNA_df.columns = depmap_mRNA_df.columns.str.replace(r" \(.*\)","")
depmap_gene_effect_df.columns = depmap_gene_effect_df.columns.str.replace(r" \(.*\)","")


C:\Users\commo\AppData\Local\Temp\ipykernel_32584\4228931511.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  depmap_gene_dependency_df.columns = depmap_gene_dependency_df.columns.str.replace(r" \(.*\)","")
C:\Users\commo\AppData\Local\Temp\ipykernel_32584\4228931511.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  depmap_mRNA_df.columns = depmap_mRNA_df.columns.str.replace(r" \(.*\)","")
C:\Users\commo\AppData\Local\Temp\ipykernel_32584\4228931511.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  depmap_gene_effect_df.columns = depmap_gene_effect_df.columns.str.replace(r" \(.*\)","")


In [13]:
# Remap Depmap ID to Cell Line Name with depmap_cells_df as a dictionary
id_cell_dict = dict(zip(cells_df['Depmap Id'], cells_df['Cell Line']))

# Remap Depmap ID to Cell Line Name with depmap_cells_df as a dictionary
depmap_gene_dependency_df['ModelID'] = depmap_gene_dependency_df['ModelID'].map(id_cell_dict)
depmap_mRNA_df['Depmap ID'] = depmap_mRNA_df['Depmap ID'].map(id_cell_dict)
depmap_gene_effect_df['Unnamed: 0'] = depmap_gene_effect_df['Unnamed: 0'].map(id_cell_dict)

C:\Users\commo\AppData\Local\Temp\ipykernel_32584\3038015071.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depmap_gene_dependency_df['ModelID'] = depmap_gene_dependency_df['ModelID'].map(id_cell_dict)
C:\Users\commo\AppData\Local\Temp\ipykernel_32584\3038015071.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depmap_mRNA_df['Depmap ID'] = depmap_mRNA_df['Depmap ID'].map(id_cell_dict)
C:\Users\commo\AppData\Local\Temp\ipykernel_32584\3038015071.py:7: SettingWithCopyWarning: 
A value is trying to be 

In [15]:
# Index to retrieve cell metadata
depmap_cells_df = cell_matches_df.iloc[:, len(cell_matches_df.columns)-5:]

In [16]:
# Sort by Cell Line Name
depmap_gene_dependency_df.sort_values(by=['ModelID'], inplace=True)
depmap_mRNA_df.sort_values(by=['Depmap ID'], inplace=True)
depmap_gene_effect_df.sort_values(by=['Unnamed: 0'], inplace=True)
depmap_cells_df.sort_values(by=['Cell Line'], inplace=True)

# Rename Columns
depmap_gene_dependency_df.rename(columns={'ModelID':'Cell Line'}, inplace=True)
depmap_mRNA_df.rename(columns={'Depmap ID':'Cell Line'}, inplace=True)
depmap_gene_effect_df.rename(columns={'Unnamed: 0':'Cell Line'}, inplace=True)


C:\Users\commo\AppData\Local\Temp\ipykernel_32584\710923112.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depmap_gene_dependency_df.sort_values(by=['ModelID'], inplace=True)
C:\Users\commo\AppData\Local\Temp\ipykernel_32584\710923112.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depmap_mRNA_df.sort_values(by=['Depmap ID'], inplace=True)
C:\Users\commo\AppData\Local\Temp\ipykernel_32584\710923112.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

In [89]:
mRNA_df.columns = depmap_mRNA_df.columns.str.replace(r" \(.*\)","")


C:\Users\commo\AppData\Local\Temp\ipykernel_22936\99317108.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  mRNA_df.columns = depmap_mRNA_df.columns.str.replace(r" \(.*\)","")


In [91]:
mRNA_df[1:].to_csv('Depmap Pancancer mRNA.csv')

#### Saves

In [83]:
# Save to CSV
depmap_cells_df.to_csv('Depmap Melanoma Cells.csv')
depmap_gene_dependency_df.to_csv('Depmap Melanoma Gene Dependency.csv')
depmap_mRNA_df.to_csv('Depmap Melanoma mRNA.csv')


In [17]:
depmap_gene_effect_df.to_csv('Depmap Melanoma Gene Effect.csv')